In [1]:
import pandas as pd
import numpy as np
import bokeh.io
import bokeh.plotting
import bokeh.palettes
import bokeh.layouts

import bokeh_catplot

from bokeh.transform import jitter
import seaborn as sns
import matplotlib
from bokeh.models import HoverTool


bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
df_long = pd.read_csv('data/TrackData.csv', sep=',', na_values='*')
df_long.head()

,Label,NUMBER_SPOTS,NUMBER_GAPS,LONGEST_GAP,NUMBER_SPLITS,NUMBER_MERGES,NUMBER_COMPLEX,TRACK_DURATION,TRACK_START,TRACK_STOP,...,TRACK_STD_SPEED,TRACK_MEAN_QUALITY,TRACK_MAX_QUALITY,TRACK_MIN_QUALITY,TRACK_MEDIAN_QUALITY,TRACK_STD_QUALITY,Strain,Reporter,TimePoint,TeckRep
0,Track_0,101,0,0,0,0,0,300,0,300,...,0.01706,2.91243,3.29013,2.38571,2.88218,0.15351,fliC-,PaQa,0,1
1,Track_1,101,0,0,0,0,0,300,0,300,...,0.01490,3.12113,3.44538,2.78503,3.10969,0.14967,fliC-,PaQa,0,1
2,Track_2,114,1,1,3,0,0,300,0,300,...,0.01475,2.75464,3.12536,2.32279,2.77496,0.18741,fliC-,PaQa,0,1
3,Track_4,104,0,0,2,0,0,300,0,300,...,0.13945,2.51595,3.83004,0.15471,2.45523,0.69808,fliC-,PaQa,0,1
4,Track_5,107,1,1,4,0,0,300,0,300,...,0.01305,2.83304,3.12536,2.55261,2.77496,0.17342,fliC-,PaQa,0,1


In [3]:
df=df_long.loc[(df_long['TRACK_DURATION']>20)]

In [4]:
df['TRACK_MEAN_SPEED_um_min']=df['TRACK_MEAN_SPEED'].multiply(60)
#df.head()

C:\Users\tala\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
Strains=list(df.Strain.unique())
TimePoints=list(df.TimePoint.unique())
Reps=list(df.TeckRep.unique())
SEM =[None]*len(Strains)*len(TimePoints)
Mean =[None]*len(Strains)*len(TimePoints)
SEM_dist =[None]*len(Strains)*len(TimePoints)
Mean_dist =[None]*len(Strains)*len(TimePoints)
Str=[None]*len(Strains)*len(TimePoints)
Time=[None]*len(Strains)*len(TimePoints)
N=[None]*len(Strains)*len(TimePoints)
n=0
for strain in Strains:
    for tp in TimePoints:
        N[n]=len(df.loc[(df['Strain'] == strain) & (df['TimePoint'] == tp),['TRACK_MEAN_SPEED_um_min']])
        Mean[n] = np.array(np.median(df.loc[(df['Strain'] == strain) & (df['TimePoint'] == tp),['TRACK_MEAN_SPEED_um_min']]))
        SEM [n] = np.array(np.std(df.loc[(df['Strain'] == strain) & (df['TimePoint'] == tp),['TRACK_MEAN_SPEED_um_min']]))
        SEM [n] = SEM [n]/np.sqrt(N[n])
        Str[n]=strain
        Time[n]=tp
        n=n+1

N=[None]*len(Strains)*len(TimePoints)
n=0
for strain in Strains:
    for tp in TimePoints:
        N[n]=len(df.loc[(df['Strain'] == strain) & (df['TimePoint'] == tp),['TRACK_DISPLACEMENT']])
        Mean_dist[n] = np.array(np.median(df.loc[(df['Strain'] == strain) & (df['TimePoint'] == tp),['TRACK_DISPLACEMENT']]))
        SEM_dist [n] = np.array(np.std(df.loc[(df['Strain'] == strain) & (df['TimePoint'] == tp),['TRACK_DISPLACEMENT']]))
        SEM_dist [n] = SEM [n]/np.sqrt(N[n])
        Str[n]=strain
        Time[n]=tp
        n=n+1

Mean_Distance_medians = df.groupby(['Strain', 'TimePoint'])['TRACK_DISPLACEMENT'].std().to_frame().reset_index()
Mean_Speed_medians = df.groupby(['Strain', 'TimePoint'])['TRACK_MEAN_SPEED_um_min'].median().to_frame().reset_index()
Summary={'SEM': SEM,
         'Median': Mean,
         'Strain': Str,
         'TimePoint': Time
        }
SEMs=pd.DataFrame.from_dict(Summary)
Mean_Distance_medians

,Strain,TimePoint,TRACK_DISPLACEMENT
0,fliC-,0,0.651248
1,fliC-,1,2.271021
2,fliC-,2,9.170755
3,fliC-,3,3.357738
4,pilG-fliC-,0,0.639125
5,pilG-fliC-,1,1.472513
6,pilG-fliC-,2,1.118631
7,pilG-fliC-,3,1.413840
8,pilH-fliC-,0,10.537491
9,pilH-fliC-,1,8.358893


In [6]:
float(SEMs.SEM[(SEMs['Strain'] == strain ) & (SEMs['TimePoint'] == tp)])
float(SEMs.Median[(SEMs['Strain'] == strain ) & (SEMs['TimePoint'] == tp)])

11.4741

In [7]:
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(
    width=600, 
    height=400,  
    x_axis_type='linear',
    x_axis_label = 'Time (s)',
    y_axis_label = 'Displacement (µm)',
    title="Track displacement"
)

p1 = bokeh.plotting.figure(
    width=600, 
    height=400,  
    x_axis_type='linear',
    x_axis_label = 'Time (s)',
    y_axis_label = 'Speed (µm/min)',
    title="Track speed"
)

In [8]:
for s, strain in enumerate(Strains):
    p.line(
        source = Mean_Distance_medians.loc[(Mean_Distance_medians['Strain'] == strain),['Strain','TimePoint','TRACK_DISPLACEMENT']],
        x = 'TimePoint',
        y = 'TRACK_DISPLACEMENT',
        color = colors[s],
        alpha=0.8,
        legend = 'Strain'
    )
    p.circle(
        source = Mean_Distance_medians.loc[(Mean_Distance_medians['Strain'] == strain),['Strain','TimePoint','TRACK_DISPLACEMENT']],
        x = 'TimePoint',
        y = 'TRACK_DISPLACEMENT',
        color = colors[s],
        alpha=0.8,
        size=5
    )
    for s, strain in enumerate(Strains):
        for tp in TimePoints:
            b = [tp,tp]
            m=float(SEMs.Median[(SEMs['Strain'] == strain ) & (SEMs['TimePoint'] == tp)])
            sem=float(SEMs.SEM[(SEMs['Strain'] == strain ) & (SEMs['TimePoint'] == tp)])
            a = [m-sem, m+sem]
            p.line(
                x = b,
                y = a,
                color = 'black',
                alpha=0.5,
                line_width=2
            )
    
for s, strain in enumerate(Strains):
    p1.line(
        source = Mean_Speed_medians.loc[(Mean_Distance_medians['Strain'] == strain),['Strain','TimePoint','TRACK_MEAN_SPEED_um_min']],
        x = 'TimePoint',
        y = 'TRACK_MEAN_SPEED_um_min',
        color = colors[s],
        alpha=0.8,
        legend = 'Strain'
    )
    p1.circle(
        source = Mean_Speed_medians.loc[(Mean_Distance_medians['Strain'] == strain),['Strain','TimePoint','TRACK_MEAN_SPEED_um_min']],
        x = 'TimePoint',
        y = 'TRACK_MEAN_SPEED_um_min',
        color = colors[s],
        alpha=0.8,
        size=5
    )

for s, strain in enumerate(Strains):
    for tp in TimePoints:
        b = [tp,tp]
        m=float(SEMs.Median[(SEMs['Strain'] == strain ) & (SEMs['TimePoint'] == tp)])
        sem=float(SEMs.SEM[(SEMs['Strain'] == strain ) & (SEMs['TimePoint'] == tp)])
        a = [m-sem, m+sem]
        p1.line(
            x = b,
            y = a,
            color = 'black',
            alpha=0.5,
            line_width=2
        )
        
p1.output_backend = 'svg'

bokeh.io.show(bokeh.layouts.row(p, p1))

In [9]:
bokeh.io.save(
    p, 
    filename='Track_displacement.html', 
    title='Track displacement'
)

bokeh.io.save(
    p1, 
    filename='Track_mean_speed.html', 
    title='Track mean speed'
)



C:\Users\tala\AppData\Local\Continuum\anaconda3\lib\site-packages\bokeh\io\saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")


'C:\\Users\\tala\\Desktop\\AnalysisMotility\\Track_mean_speed.html'

In [10]:
p = bokeh_catplot.ecdf(
    data=df.loc[(df['Strain'] == 'fliC-_PaQa'),['TimePoint', 'TRACK_DISPLACEMENT', 'TRACK_MEAN_SPEED']],
    cats='TimePoint',
    val='TRACK_DISPLACEMENT',
    formal=True,
    conf_int=True,
    title='fliC- PaQa'
)
p1 = bokeh_catplot.ecdf(
    data=df.loc[(df['Strain'] == 'fliC-_PaQa'),['TimePoint', 'TRACK_DISPLACEMENT', 'TRACK_MEAN_SPEED']],
    cats='TimePoint',
    val='TRACK_MEAN_SPEED',
    formal=True,
    conf_int=True,
    title='fliC- PaQa'
)
bokeh.io.show(bokeh.layouts.row(p, p1))

C:\Users\tala\AppData\Local\Continuum\anaconda3\lib\site-packages\bokeh\models\plots.py:756: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with the 'legend' parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
C:\Users\tala\AppData\Local\Continuum\anaconda3\lib\site-packages\bokeh\models\plots.py:756: UserWarning: 
You are attempting to set `plot.legend.click_policy` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with the 'legend' parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)


In [11]:
p = bokeh_catplot.ecdf(
    data=df.loc[(df['Strain'] == 'pilG-fliC-_PaQa'),['TimePoint', 'TRACK_DISPLACEMENT', 'TRACK_MEAN_SPEED']],
    cats='TimePoint',
    val='TRACK_DISPLACEMENT',
    formal=True,
    conf_int=True,
    title='pilG-fliC- PaQa'
)
p1 = bokeh_catplot.ecdf(
    data=df.loc[(df['Strain'] == 'pilG-fliC-_PaQa'),['TimePoint', 'TRACK_DISPLACEMENT', 'TRACK_MEAN_SPEED']],
    cats='TimePoint',
    val='TRACK_MEAN_SPEED',
    formal=True,
    conf_int=True,
    title='pilG-fliC- PaQa'
)
bokeh.io.show(bokeh.layouts.row(p, p1))

In [12]:
p = bokeh_catplot.ecdf(
    data=df.loc[(df['Strain'] == 'pilH-fliC-_PaQa'),['TimePoint', 'TRACK_DISPLACEMENT', 'TRACK_MEAN_SPEED']],
    cats='TimePoint',
    val='TRACK_DISPLACEMENT',
    formal=True,
    conf_int=True,
    title='pilH-fliC- PaQa'
)
p1 = bokeh_catplot.ecdf(
    data=df.loc[(df['Strain'] == 'pilH-fliC-_PaQa'),['TimePoint', 'TRACK_DISPLACEMENT', 'TRACK_MEAN_SPEED']],
    cats='TimePoint',
    val='TRACK_MEAN_SPEED',
    formal=True,
    conf_int=True,
    title='pilH-fliC- PaQa'
)
bokeh.io.show(bokeh.layouts.row(p, p1))

In [13]:
p = bokeh_catplot.ecdf(
    data=df,
    cats='Strain',
    val='TRACK_DISPLACEMENT',
    formal=True,
    conf_int=True
)
p1 = bokeh_catplot.ecdf(
    data=df,
    cats='Strain',
    val='TRACK_MEAN_SPEED',
    formal=True,
    conf_int=True
)
bokeh.io.show(bokeh.layouts.row(p, p1))